# Bank Payment Processor

This notebook helps you process bank statements, match transactions to members, and export results for review. Please follow the steps in order.

## Step 1: Configure Import Settings

Set up the parameters for importing your bank statement:

- **File Path**: Full path to the Excel file containing the bank statement
- **Sheet**: Sheet name or index (0 = first sheet)
- **Column Names**: Specify which columns contain the transaction date, type, description, and amount
- **Minimum Amount**: Only transactions with an amount greater than this value will be processed

Use the widgets below to enter your settings.

In [21]:
# 2. Django environment and imports
# Setup: Import Django models and configure the environment

# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import re
from datetime import datetime
from decimal import Decimal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
import threading
import django

# Required for Excel export with formatting
try:
    import xlsxwriter
except ImportError:
    print("⚠️ xlsxwriter module not found. Installing...")
    import pip
    pip.main(['install', 'xlsxwriter'])
    import xlsxwriter
    print("✅ xlsxwriter installed successfully")



# Helper function for threading (moved up from the helper cell)
def run_in_thread(func):
    """Execute a function in a thread to avoid async issues with Django"""
    import threading
    result = []
    error = []
    
    def wrapped_func():
        try:
            result.append(func())
        except Exception as e:
            error.append(e)
    
    thread = threading.Thread(target=wrapped_func)
    thread.start()
    thread.join()
    
    if error:
        raise error[0]
    
    return result[0] if result else None

# Add project directory to path and set up Django
project_dir = r'C:/Work/active_projects/web-based-membership-management-system'
if project_dir not in sys.path:
    sys.path.append(project_dir)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'yzer_membership.settings')
django.setup()

# Import Django models and query tools
from members.models import Member, PaymentRecord, BankStatementImport
from django.db.models import Q

print("✅ Django environment set up successfully")
print(f"✅ Connected to database: {Member._meta.app_config.name}")
print(f"✅ Project directory: {project_dir}")
print(f"✅ Input directory: {os.path.join(project_dir, 'data_input')}")
print(f"✅ Output directory: {os.path.join(project_dir, 'data_output')}")

# For demo purposes, display a count of members in the database using a thread
def get_counts():
    return (
        Member.objects.count(),
        PaymentRecord.objects.count()
    )

member_count, payment_count = run_in_thread(get_counts)
print(f"📊 Database contains {member_count} members and {payment_count} payment records")

# --- File and column settings for loading bank statement ---
file_path = r'C:/Work/active_projects/web-based-membership-management-system/data_input/statement-08-240-533-6 Feb-July.xlsx'
sheet = 'Sheet1'  # Sheet name as provided
date_col = 'Date'  # Column name for transaction date
type_col = 'Type'  # Column name for transaction type
description_col = 'Description'  # Column name for transaction description
amount_col = 'Amount'  # Column name for transaction amount
min_amount = 50.0  # Minimum amount filter

✅ Django environment set up successfully
✅ Connected to database: members
✅ Project directory: C:/Work/active_projects/web-based-membership-management-system
✅ Input directory: C:/Work/active_projects/web-based-membership-management-system\data_input
✅ Output directory: C:/Work/active_projects/web-based-membership-management-system\data_output
📊 Database contains 2049 members and 82 payment records


In [22]:
# 3. Utility and matching functions
# Utility functions for data processing and member matching
from typing import Optional, List, Dict, Any
from decimal import Decimal
import pandas as pd
import re
from django.db.models import Q
from fuzzywuzzy import fuzz
import ipywidgets as widgets
from IPython.display import display

# Define the priority order for membership status (used for both export and matching)
status_priority = {
    'Paid Up Member': 1,
    'Unpaid Member': 2,
    'Subscribed': 3,
    'Contact': 4,
    'Cleaned': 5,
    'Pending': 6,
    'WhatsApp': 7
}

# Define the priority order for membership_type (for third sort level)
membership_type_priority = {
    'member': 1,
    'life_member': 2,
    'honorary_member': 3,
    'MailChimp Audience': 4,
    'Contact': 5,
    'Yzer Conservancy': 6,
    'Yzer Residents Association - Information': 7,
    'Concerned Residents': 8
}

def clean_amount(amount_str: Any) -> Decimal:
    """Convert amount to Decimal"""
    if pd.isna(amount_str) or amount_str is None or amount_str == "":
        return Decimal('0.00')
    if isinstance(amount_str, (int, float)):
        return Decimal(str(amount_str))
    cleaned = re.sub(r'[R\s,]', '', str(amount_str))
    try:
        return Decimal(cleaned)
    except Exception as e:
        return Decimal('0.00')

class MemberMatcher:
    """Class to handle member matching logic with strict priority grouping"""
    
    def __init__(self):
        self.member_cache = {}
        self.status_priority = status_priority
        self.membership_type_priority = membership_type_priority
    
    def normalize_name(self, text: str) -> str:
        """Normalize a name for comparison"""
        if not text:
            return ""
        text = text.lower()
        text = text.split(',')[0]
        text = re.sub(r'[^a-z0-9\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def get_members(self) -> List[Dict]:
        """Get all members from database with caching, sorted by status and type priority and then name"""
        if not self.member_cache:
            members = run_in_thread(lambda: list(Member.objects.all().values(
                "id", "first_name", "last_name", "membership_status", "email", "membership_type"
            )))
            members = [m for m in members if m.get('first_name') and m.get('last_name')]
            for m in members:
                m['status_priority'] = self.status_priority.get(m.get('membership_status'), 99)
                m['type_priority'] = self.membership_type_priority.get(m.get('membership_type'), 99)
                m['full_name'] = f"{m['first_name']} {m['last_name']}"
            members = sorted(members, key=lambda m: (m['status_priority'], m['type_priority'], m['full_name']))
            self.member_cache = {m['id']: m for m in members}
        return [self.member_cache[k] for k in self.member_cache]

    def match_transaction(self, txn_desc: str) -> Optional[Dict]:
        """Strictly match a transaction description to a member, searching only in the highest priority group first"""
        txn_desc = self.normalize_name(txn_desc)
        members = self.get_members()
        
        # Group members by (status_priority, type_priority)
        from collections import defaultdict
        grouped = defaultdict(list)
        for m in members:
            key = (m['status_priority'], m['type_priority'])
            grouped[key].append(m)
        
        # Sort groups by priority (lowest first)
        sorted_keys = sorted(grouped.keys())
        
        for key in sorted_keys:
            group = grouped[key]
            best_match = None
            best_score = 0
            for member in group:
                last_name = self.normalize_name(member['last_name'])
                first_name = self.normalize_name(member['first_name'])
                
                # Try exact last name match first
                if last_name in txn_desc:
                    score = 90  # Base score for last name match
                    if first_name in txn_desc:
                        score += 10
                    if member['status_priority'] == 1:
                        score += 50
                    if score > best_score:
                        best_score = score
                        best_match = member
            if best_match:
                return best_match  # Only return if found in this highest-priority group
        return None

# Create global matcher instance
member_matcher = MemberMatcher()

print("✅ Utility and matching functions cell completed OK (strict priority matching)")

✅ Utility and matching functions cell completed OK (strict priority matching)


In [29]:
# 4. Import bank account transactions

import os
import pandas as pd
import datetime

def load_and_summarise_statements(export_excel=True):
    """
    Load all statement Excel files, combine into a single DataFrame, print summary,
    and optionally export to Excel.
    """
    input_dir = os.path.join(project_dir, 'data_input')
    file_pattern = os.path.join(input_dir, 'statement*.xlsx')

    import glob
    files = glob.glob(file_pattern)
    print(f"Found {len(files)} statement files: {[os.path.basename(f) for f in files]}")

    all_records = []
    for file_path in files:
        try:
            print(f"\nReading file: {os.path.basename(file_path)}")
            df = pd.read_excel(
                file_path,
                header=None,
                usecols="B,D,E,F",
                skiprows=3,
                engine='openpyxl'
            )
            df.columns = ['Date', 'Amount', 'Type', 'Description']
            all_records.append(df)
            print(f"✅ Loaded {len(df)} rows from {os.path.basename(file_path)}")
        except Exception as e:
            print(f"❌ Error loading {file_path}: {str(e)}")

    if not all_records:
        print("❌ No records to process.")
        return None

    combined_df = pd.concat(all_records, ignore_index=True)
    print(f"\nTotal combined records: {len(combined_df)}")
    print("Columns:", list(combined_df.columns))
    print("\nSample data:")
    #print(combined_df.head())

    if export_excel:
        output_dir = os.path.join(project_dir, 'data_output')
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(output_dir, f'combined_statements_{timestamp}.xlsx')
        try:
            combined_df.to_excel(output_path, index=False)
            print(f"✅ Exported combined DataFrame to: {output_path}")
            os.startfile(output_path)  # This will open the file automatically on Windows
        except Exception as e:
            print(f"❌ Failed to export Excel file: {str(e)}")
    else:
        print("ℹ️ Export suppressed (export_excel=False)")

    return combined_df

# Call the function to load, summarise, and export
df = load_and_summarise_statements(export_excel=False)
if df is not None and len(df) > 0:
    out = widgets.Output()
    with out:
        display(df.head(5).style.set_table_attributes('style="display:inline"').set_caption("First 5 Records"))
    display(widgets.VBox([widgets.HTML("<b>First 5 Records (scrollable):</b>"), widgets.Box([out], layout=widgets.Layout(overflow='auto', max_height='200px'))]))
else:
    print("No data to display.")


Found 7 statement files: ['statement-08-240-533-6 20240828.xlsx', 'statement-08-240-533-6 20250218.xlsx', 'statement-08-240-533-6 20250310.xlsx', 'statement-08-240-533-6 20250317.xlsx', 'statement-08-240-533-6 20250424.xlsx', 'statement-08-240-533-6 20250704.xlsx', 'statement-08-240-533-6 20250725.xlsx']

Reading file: statement-08-240-533-6 20240828.xlsx
✅ Loaded 61 rows from statement-08-240-533-6 20240828.xlsx

Reading file: statement-08-240-533-6 20250218.xlsx
✅ Loaded 37 rows from statement-08-240-533-6 20250218.xlsx

Reading file: statement-08-240-533-6 20250310.xlsx
✅ Loaded 84 rows from statement-08-240-533-6 20250310.xlsx

Reading file: statement-08-240-533-6 20250317.xlsx
✅ Loaded 24 rows from statement-08-240-533-6 20250317.xlsx

Reading file: statement-08-240-533-6 20250424.xlsx
✅ Loaded 19 rows from statement-08-240-533-6 20250424.xlsx

Reading file: statement-08-240-533-6 20250704.xlsx
✅ Loaded 11 rows from statement-08-240-533-6 20250704.xlsx

Reading file: statement-08-

In [24]:
# 5. Clean the imported data
import os
import pandas as pd
from datetime import datetime

def clean_and_export_data(df, export_excel=False):
    """
    Clean the DataFrame:
    1. Convert 'Date' column to datetime.
    2. Filter out rows where 'Amount' < 0.
    3. Optionally export and open the cleaned data with a timestamped filename.
    """
    # 1. Convert 'Date' to datetime (robust conversion)
    def convert_any_date(val):
        if pd.isna(val):
            return None
        # Handle YYYYMMDD as int or str
        if (isinstance(val, int) or (isinstance(val, float) and val.is_integer())) and len(str(int(val))) == 8:
            try:
                return datetime.strptime(str(int(val)), '%Y%m%d')
            except Exception:
                return None
        if isinstance(val, str) and len(val) == 8 and val.isdigit():
            try:
                return datetime.strptime(val, '%Y%m%d')
            except Exception:
                return None
        # Excel serial date
        if isinstance(val, (float, int)):
            try:
                return pd.to_datetime(val, unit='d', origin='1899-12-30')
            except Exception:
                return None
        # Try parse as string date
        if isinstance(val, str):
            for fmt in ('%Y-%m-%d', '%d/%m/%Y', '%d-%m-%Y', '%Y/%m/%d', '%m/%d/%Y'):
                try:
                    return datetime.strptime(val, fmt)
                except Exception:
                    continue
            try:
                return pd.to_datetime(val, errors='coerce')
            except Exception:
                return None
        return None

    df['Date'] = df['Date'].apply(convert_any_date)
    df = df[df['Date'].notna()].copy()  # <-- Use .copy() here to avoid SettingWithCopyWarning
    df['Date'] = df['Date'].apply(lambda d: d.strftime('%Y-%m-%d') if pd.notna(d) else '')

    # 2. Filter out rows where 'Amount' < 0
    df = df[df['Amount'] >= 0]

    print(f"Cleaned records: {len(df)}")
    #print(df.head())

    # 3. Export and open if requested
    if export_excel:
        output_dir = os.path.join(project_dir, 'data_output')
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(output_dir, f'cleaned_data_{timestamp}.xlsx')
        try:
            df.to_excel(output_path, index=False)
            print(f"✅ Exported cleaned DataFrame to: {output_path}")
            os.startfile(output_path)  # Open file on Windows
        except Exception as e:
            print(f"❌ Failed to export Excel file: {str(e)}")
    else:
        print("ℹ️ Export suppressed (export_excel=False)")

    return df

# Example usage:
# cleaned_df = clean_and_export_data(df, export_excel
# Call the function to load, summarise, and export
df = clean_and_export_data(df, export_excel=False)
if df is not None and len(df) > 0:
    out = widgets.Output()
    with out:
        display(df.head(5).style.set_table_attributes('style="display:inline"').set_caption("First 5 Records"))
    display(widgets.VBox([widgets.HTML("<b>First 5 Records (scrollable):</b>"), widgets.Box([out], layout=widgets.Layout(overflow='auto', max_height='200px'))]))
else:
    print("No data to display.")


Cleaned records: 186
ℹ️ Export suppressed (export_excel=False)


In [32]:
# 6 Inspect the Django member table data

import os
import pandas as pd
from datetime import datetime
import pytz
import ipywidgets as widgets
from IPython.display import display


def export_all_members_to_excel(export_excel=False, auto_open=True):
    """
    Export all records from the Django Member table to Excel.
    Parameters:
        export_excel (bool): If True, export to Excel. If False, just return the DataFrame.
        auto_open (bool): If True, open the file after export (Windows only).
    Returns:
        members_df (pd.DataFrame): DataFrame of all members.
    """
    # Query all members as dictionaries
    members = run_in_thread(lambda: list(Member.objects.all().values()))
    members_df = pd.DataFrame(members)
    members_df['full_name'] = members_df['first_name'].fillna('') + ' ' + members_df['last_name'].fillna('')

    print(f"Total members: {len(members_df)}")
    #print(members_df.head())

     # Make all datetime columns timezone-unaware for Excel export
    for col in members_df.select_dtypes(include=['datetimetz']).columns:
        members_df[col] = members_df[col].dt.tz_localize(None)
    if export_excel:
        output_dir = os.path.join(project_dir, 'data_output')
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(output_dir, f'all_members_{timestamp}.xlsx')
        try:
            members_df.to_excel(output_path, index=False)
            print(f"✅ Exported results to: {output_path}")
            # Standardize output path display
            rel_output = output_path.replace('\\', '/')
            marker = '/web-based-membership-management-system/data_output/'
            idx = rel_output.find(marker)
            if idx != -1:
                rel_output = rel_output[idx + 1:] if rel_output[idx] == '/' else rel_output[idx:]
            print(f"📁 File saved: {rel_output}")
            if auto_open:
                os.startfile(output_path)
            if auto_open:
                os.startfile(output_path)
        except Exception as e:
            print(f"❌ Failed to export Excel file: {str(e)}")
    else:
        print("ℹ️ Export suppressed (export_excel=False)")

    return members_df

# Example usage:
members_df = export_all_members_to_excel(export_excel=False, auto_open=True)

if members_df is not None and len(members_df) > 0:
    out = widgets.Output()
    with out:
        display(members_df.head(5).style.set_table_attributes('style="display:inline"').set_caption("First 5 Members"))
    display(widgets.VBox([widgets.HTML("<b>First 5 Members (scrollable):</b>"), widgets.Box([out], layout=widgets.Layout(overflow='auto', max_height='200px'))]))
else:
    print("No data to display.")
# members_df = export_all_members_to_excel(export_excel=True)

Total members: 2049
ℹ️ Export suppressed (export_excel=False)


In [33]:
# 7 --- Staged, priority-based member matching function ---

from fuzzywuzzy import fuzz

def staged_member_match(desc, members, status_priority):
    """
    Try to match a transaction description to a member in stages:
    1. Both first_name and last_name present, by status priority
    2. Only last_name present, by status priority
    3. Only first_name present, by status priority
    Returns: (match_dict, score) or (None, 0)
    """
    desc = str(desc).lower().strip()
    # Stage 1: Both names present
    for status in sorted(status_priority, key=lambda k: status_priority[k]):
        candidates = [m for m in members if m.get('membership_status') == status and m.get('first_name') and m.get('last_name')]
        best_match, best_score = None, 0
        for m in candidates:
            full_name = f"{m['first_name']} {m['last_name']}".strip().lower()
            score = max(
                fuzz.token_set_ratio(full_name, desc),
                fuzz.partial_ratio(full_name, desc),
                fuzz.ratio(full_name, desc)
            )
            # Bonus logic
            last_name = m['last_name'].strip().lower()
            first_name = m['first_name'].strip().lower()
            has_last = last_name in desc
            has_first = first_name in desc
            if has_last and has_first:
                score += 15
            elif has_last:
                score += 10
            elif has_first:
                score += 2
            score = min(score, 100)
            if score > best_score:
                best_score = score
                best_match = m
        if best_score >= 85:
            return best_match, best_score

    # Stage 2: Only last_name present
    for status in sorted(status_priority, key=lambda k: status_priority[k]):
        candidates = [m for m in members if m.get('membership_status') == status and m.get('last_name') and not m.get('first_name')]
        best_match, best_score = None, 0
        for m in candidates:
            last_name = m['last_name'].strip().lower()
            score = max(
                fuzz.token_set_ratio(last_name, desc),
                fuzz.partial_ratio(last_name, desc),
                fuzz.ratio(last_name, desc)
            )
            if last_name in desc:
                score += 10
            score = min(score, 100)
            if score > best_score:
                best_score = score
                best_match = m
        if best_score >= 85:
            return best_match, best_score

    # Stage 3: Only first_name present
    for status in sorted(status_priority, key=lambda k: status_priority[k]):
        candidates = [m for m in members if m.get('membership_status') == status and m.get('first_name') and not m.get('last_name')]
        best_match, best_score = None, 0
        for m in candidates:
            first_name = m['first_name'].strip().lower()
            score = max(
                fuzz.token_set_ratio(first_name, desc),
                fuzz.partial_ratio(first_name, desc),
                fuzz.ratio(first_name, desc)
            )
            if first_name in desc:
                score += 2
            score = min(score, 100)
            if score > best_score:
                best_score = score
                best_match = m
        if best_score >= 85:
            return best_match, best_score

    # No match found
    return None, 0

print("✅ Staged member matching function ready for use.")

✅ Staged member matching function ready for use.


In [ ]:
# 8 --- Fuzzy Logic Matching, priority-based member matching function ---
# Main processing and export function (updated for full output columns and logic)
# Using Fuzzy Logic Matching
import os
import pandas as pd
from datetime import datetime
from decimal import Decimal
from fuzzywuzzy import fuzz
import re

print(df.shape)
print(df.head())

def process_transactions(df, export_excel=True, auto_open=True):
    """
    Process bank transactions, match Description to member full name, and export results to Excel with all required columns and logic.
    Args:
        df: DataFrame of cleaned bank transactions (already loaded and cleaned)
    Returns:
        matched_df: DataFrame of matched transactions
        unmatched_df: DataFrame of unmatched transactions
    """
    print("Function started")

    if df is None or len(df) == 0:
        print("❌ No data provided to process_transactions.")
        return None, None

    # --- Prepare member names for matching ---
    members = run_in_thread(lambda: list(Member.objects.all().values(
        'id','member_id','first_name', 'last_name', 'membership_status', 'email', 'membership_type'
    )))
    print(f"\nTotal members loaded: {len(members)}")
    for m in members:
        m['full_name'] = f"{m['first_name']} {m['last_name']}".strip().lower()

    # --- Set cutoff date for Paid status ---
    # EDIT THIS DATE FOR DETERMINING IF PAYMENT IS DUE OR NOT
    paid_cutoff_date = datetime(2025, 1, 1).date()

    # --- Match transactions to members by Description ---
    matched_rows = []
    unmatched_rows = []

    # THIS IS WHERE WE MATCH TRANSACTIONS TO MEMBERS....
    for idx, row in df.iterrows():
        desc = str(row['Description']).lower() if pd.notna(row['Description']) else ''
        match, best_score = staged_member_match(desc, members, status_priority)
    
    # THIS IS WHERE WE DETERMINE IF WE HAVE A MATCH
        if match:
            last_payment = run_in_thread(lambda: PaymentRecord.objects.filter(
                member_id=match['id']
            ).order_by('-payment_date').first())
            last_payment_date = last_payment.payment_date if last_payment else None
            last_payment_amount = last_payment.amount if last_payment else None
            is_paid = False
            if last_payment_date and last_payment_amount:
                try:
                    is_paid = (last_payment_date >= paid_cutoff_date) and (Decimal(last_payment_amount) > 0)
                except Exception as e:
                    print(f"Error determining payment status: {e}")
                    is_paid = False
            # --- Compare Amount and Last_Payment_Amount and Dates ---
            try:
                amt = Decimal(str(row['Amount']))
                last_amt = Decimal(str(last_payment_amount)) if last_payment_amount is not None else None
                txn_date = pd.to_datetime(row['Date']).date() if pd.notna(row['Date']) else None
                last_pay_date = pd.to_datetime(last_payment_date).date() if last_payment_date is not None else None

                amount_match = (last_amt is not None and amt == last_amt)
                date_match = (txn_date is not None and last_pay_date is not None and txn_date == last_pay_date)

                # Combinations
                if amount_match and date_match:
                    result = 'Correct Amount & Date'
                elif amount_match:
                    result = 'Correct Amount'
                elif date_match:
                    result = 'Correct Date'
                elif last_payment_date is None:
                    result = 'Update Payment'
                else:
                    result = 'Paid' if is_paid else 'Unpaid'
            except Exception as e:
                print(f"Error comparing amounts/dates: {e}")
                result = 'Paid' if is_paid else 'Unpaid'
            # --- Append matched row ---
            clean_name = re.sub(r'[^a-zA-Z0-9 ]', '', f"{match['first_name']} {match['last_name']}")
            matched_rows.append({
                'Type': row['Type'],
                'Transaction_Date': row['Date'],
                'Description': row['Description'],
                'Amount': row['Amount'],
                'Match_Score': best_score,
                'Result': result,
                'Matched_Member_ID': match['member_id'],
                'Matched_Member_Name': clean_name,
                'Membership_Status': match['membership_status'],
                'Matched_Member_Email': match['email'],
                'Last_Payment_Date': last_payment_date,
                'Last_Payment_Amount': last_payment_amount,
                'id': match['id']
            })
        else:
            unmatched_rows.append({
                'Type': row['Type'],
                'Transaction_Date': row['Date'],
                'Description': row['Description'],
                'Amount': row['Amount'],
                'Match_Score': '',
                'Result': '',
                'Matched_Member_ID': '',
                'Matched_Member_Name': '',
                'Membership_Status': '',
                'Matched_Member_Email': '',
                'Last_Payment_Date': '',
                'Last_Payment_Amount': ''
            })

    matched_df = pd.DataFrame(matched_rows)
    unmatched_df = pd.DataFrame(unmatched_rows)

    print(f"\nProcessing results:")
    print(f"Matched: {len(matched_df)} | Unmatched: {len(unmatched_df)}")

    # --- Export to Excel ---
    if len(matched_df) > 0 or len(unmatched_df) > 0:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_dir = os.path.join(project_dir, 'data_output')
        os.makedirs(output_dir, exist_ok=True)
        # THIS IS WHERE THE FILE NAME CAN BE CHANGED IN THE EXPORTED EXCEL FILE
        output_path = os.path.join(output_dir, f'transaction_matching_staged_{timestamp}.xlsx')
        try:
            with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
                # Specify columns to export for each sheet
                matched_cols = [
                    'Type', 'Transaction_Date', 'Description', 'Amount', 'Match_Score', 'Result',
                    'Matched_Member_ID', 'Matched_Member_Name', 'Membership_Status',
                    'Matched_Member_Email', 'Last_Payment_Date', 'Last_Payment_Amount', 'id'
                ]
                unmatched_cols = [
                    'Type', 'Transaction_Date', 'Description', 'Amount'
                ]
                members_cols = [
                    'id', 'first_name', 'last_name',  'full_name','email', 'cell_phone', 'membership_status', 'membership_type',
                    'date_paid','payment_received','member_id','source_file','mailchimp_id','last_synced'
                ]
                # Only export the selected columns from above
                if len(matched_df) > 0:
                    # Ensure Amount and Last_Payment_Amount are numeric for formatting to work
                    matched_df['Amount'] = pd.to_numeric(matched_df['Amount'], errors='coerce')
                    matched_df['Last_Payment_Amount'] = pd.to_numeric(matched_df['Last_Payment_Amount'], errors='coerce')
                    matched_df[matched_cols].to_excel(writer, sheet_name='Matched Transactions', index=False)

                if len(unmatched_df) > 0:
                    unmatched_df[unmatched_cols].to_excel(writer, sheet_name='Unmatched Transactions', index=False)
                
                if 'members_df' in globals() and len(members_df) > 0:
                    # Add status priority column for custom sorting
                    status_priority_map = {
                        'Paid Up Member': 1,
                        'Unpaid Member': 2,
                        'Subscribed': 3,
                        'Contact': 4,
                        'Cleaned': 5,
                        'Pending': 6,
                        'WhatsApp': 7
                    }
                    # Change the sort order here
                    members_df['status_priority'] = members_df['membership_status'].map(status_priority_map).fillna(99)
                    # Sort by full_name (A-Z), then by status_priority (lowest number = highest priority)
                    members_df_sorted = members_df.sort_values(
                        by=['full_name', 'status_priority'],
                        ascending=[True, True]
                    )
                    members_df_sorted[members_cols].to_excel(writer, sheet_name='All Members', index=False)
                
                workbook = writer.book
                # --- Add currency formatting ---
                currency_format = workbook.add_format({'num_format': 'R #,##0.00'})
                # Format 'Amount' and 'Last_Payment_Amount' in Matched Transactions
                if 'Matched Transactions' in writer.sheets:
                    ws = writer.sheets['Matched Transactions']
                    amt_col = matched_cols.index('Amount')
                    last_amt_col = matched_cols.index('Last_Payment_Amount')
                    ws.set_column(amt_col, amt_col, 12, currency_format)
                    ws.set_column(last_amt_col, last_amt_col, 18, currency_format)
                # Format 'Amount' in Unmatched Transactions
                if 'Unmatched Transactions' in writer.sheets:
                    ws = writer.sheets['Unmatched Transactions']
                    amt_col = unmatched_cols.index('Amount')
                    ws.set_column(amt_col, amt_col, 12, currency_format)
                # Format 'Last_Payment_Amount' in All Members (if present)
                if 'All Members' in writer.sheets and 'Last_Payment_Amount' in members_cols:
                    ws = writer.sheets['All Members']
                    last_amt_col = members_cols.index('Last_Payment_Amount')
                    ws.set_column(last_amt_col, last_amt_col, 18, currency_format)
                # Autofilter and freeze panes
                for sheet in writer.sheets:
                    worksheet = writer.sheets[sheet]
                    worksheet.autofilter(0, 0, worksheet.dim_rowmax, worksheet.dim_colmax)
                    worksheet.freeze_panes(1, 0)
            print(f"✅ Exported results to: {output_path}")
            
            # Standardize output path display
            rel_output = output_path.replace('\\', '/')
            marker = '/web-based-membership-management-system/data_output/'
            idx = rel_output.find(marker)
            if idx != -1:
                rel_output = rel_output[idx + 1:] if rel_output[idx] == '/' else rel_output[idx:]
            print(f"📁 File saved: {rel_output}")
            if auto_open:
                os.startfile(output_path)

        except Exception as e:
            print(f"❌ Failed to export Excel file: {str(e)}")

    return matched_df, unmatched_df

# Usage example:
matched_df, unmatched_df = process_transactions(df, export_excel=True, auto_open=True)

(240, 4)
       Date  Amount                    Type                     Description
0  20250131   200.0         IB PAYMENT FROM            COGHO - OCTOBER 2023
1  20250131   -46.7             SERVICE FEE                             NaN
2  20250131   -95.0  MONTHLY MANAGEMENT FEE                             NaN
3  20250203  -149.0         ACCOUNT PAYMENT  MULTID FORXNEELO      85541122
4  20250205 -1044.3           IB PAYMENT TO             BEN TROMP ONDERHOUD
Function started

Total members loaded: 2049

Processing results:
Matched: 39 | Unmatched: 201
✅ Exported results to: C:/Work/active_projects/web-based-membership-management-system\data_output\transaction_matching_staged_20250809_143010.xlsx
📁 File saved: web-based-membership-management-system/data_output/transaction_matching_staged_20250809_143010.xlsx


In [36]:
# 9 --- Simple, priority-based member matching function ---
# Main processing and export function (updated for full output columns and logic
# Using Simple Name Matching
import os
import pandas as pd
from datetime import datetime
from decimal import Decimal
from fuzzywuzzy import fuzz
import re

print(df.shape)
print(df.head())

def process_transactions(df, export_excel=True, auto_open=True):
    """
    Process bank transactions, match Description to member full name, and export results to Excel with all required columns and logic.
    Args:
        df: DataFrame of cleaned bank transactions (already loaded and cleaned)
    Returns:
        matched_df: DataFrame of matched transactions
        unmatched_df: DataFrame of unmatched transactions
    """
    print("Function started")

    if df is None or len(df) == 0:
        print("❌ No data provided to process_transactions.")
        return None, None

    # --- Prepare member names for matching ---
    members = run_in_thread(lambda: list(Member.objects.all().values(
        'id','member_id','first_name', 'last_name', 'membership_status', 'email', 'membership_type'
    )))
    print(f"\nTotal members loaded: {len(members)}")
    for m in members:
        m['full_name'] = f"{m['first_name']} {m['last_name']}".strip().lower()

   # --- Set cutoff date for Paid status ---
    # EDIT THIS DATE FOR DETERMINING IF PAYMENT IS DUE OR NOT
    paid_cutoff_date = datetime(2025, 1, 1).date()

    # --- Match transactions to members by Description ---
    matched_rows = []
    unmatched_rows = []

    # THIS IS WHERE WE MATCH TRANSACTIONS TO MEMBERS....
    for idx, row in df.iterrows():
        desc = str(row['Description']).lower() if pd.notna(row['Description']) else ''
        #match, best_score = staged_member_match(desc, members, status_priority)
        match = None
        best_score = 0
        for m in members:
            if m['full_name']:
                score = fuzz.token_set_ratio(m['full_name'], desc)
                if score > best_score:
                    best_score = score
                    match = m
        if best_score < 85:
            match = None

    # THIS IS WHERE WE DETERMINE IF WE HAVE A MATCH
        if match:
            last_payment = run_in_thread(lambda: PaymentRecord.objects.filter(
                member_id=match['id']
            ).order_by('-payment_date').first())
            last_payment_date = last_payment.payment_date if last_payment else None
            last_payment_amount = last_payment.amount if last_payment else None
            is_paid = False
            if last_payment_date and last_payment_amount:
                try:
                    is_paid = (last_payment_date >= paid_cutoff_date) and (Decimal(last_payment_amount) > 0)
                except Exception as e:
                    print(f"Error determining payment status: {e}")
                    is_paid = False
            # --- Compare Amount and Last_Payment_Amount and Dates ---
            try:
                amt = Decimal(str(row['Amount']))
                last_amt = Decimal(str(last_payment_amount)) if last_payment_amount is not None else None
                txn_date = pd.to_datetime(row['Date']).date() if pd.notna(row['Date']) else None
                last_pay_date = pd.to_datetime(last_payment_date).date() if last_payment_date is not None else None

                amount_match = (last_amt is not None and amt == last_amt)
                date_match = (txn_date is not None and last_pay_date is not None and txn_date == last_pay_date)

                # Combinations
                if amount_match and date_match:
                    result = 'Correct Amount & Date'
                elif amount_match:
                    result = 'Correct Amount'
                elif date_match:
                    result = 'Correct Date'
                elif last_payment_date is None:
                    result = 'Update Payment'
                else:
                    result = 'Paid' if is_paid else 'Unpaid'
            except Exception as e:
                print(f"Error comparing amounts/dates: {e}")
                result = 'Paid' if is_paid else 'Unpaid'
            # --- Append matched row ---
            clean_name = re.sub(r'[^a-zA-Z0-9 ]', '', f"{match['first_name']} {match['last_name']}")
            matched_rows.append({
                'Type': row['Type'],
                'Transaction_Date': row['Date'],
                'Description': row['Description'],
                'Amount': row['Amount'],
                'Match_Score': best_score,
                'Result': result,
                'Matched_Member_ID': match['member_id'],
                'Matched_Member_Name': clean_name,
                'Membership_Status': match['membership_status'],
                'Matched_Member_Email': match['email'],
                'Last_Payment_Date': last_payment_date,
                'Last_Payment_Amount': last_payment_amount,
                'id': match['id']
            })
        else:
            unmatched_rows.append({
                'Type': row['Type'],
                'Transaction_Date': row['Date'],
                'Description': row['Description'],
                'Amount': row['Amount'],
                'Match_Score': '',
                'Result': '',
                'Matched_Member_ID': '',
                'Matched_Member_Name': '',
                'Membership_Status': '',
                'Matched_Member_Email': '',
                'Last_Payment_Date': '',
                'Last_Payment_Amount': ''
            })

    matched_df = pd.DataFrame(matched_rows)
    unmatched_df = pd.DataFrame(unmatched_rows)

    print(f"\nProcessing results:")
    print(f"Matched: {len(matched_df)} | Unmatched: {len(unmatched_df)}")

    # --- Export to Excel ---
    if len(matched_df) > 0 or len(unmatched_df) > 0:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_dir = os.path.join(project_dir, 'data_output')
        os.makedirs(output_dir, exist_ok=True)
        # THIS IS WHERE THE FILE NAME CAN BE CHANGED IN THE EXPORTED EXCEL FILE
        output_path = os.path.join(output_dir, f'transaction_matching_simple_{timestamp}.xlsx')
        try:
            with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
                # Specify columns to export for each sheet
                matched_cols = [
                    'Type', 'Transaction_Date', 'Description', 'Amount', 'Match_Score', 'Result',
                    'Matched_Member_ID', 'Matched_Member_Name', 'Membership_Status',
                    'Matched_Member_Email', 'Last_Payment_Date', 'Last_Payment_Amount', 'id'
                ]
                unmatched_cols = [
                    'Type', 'Transaction_Date', 'Description', 'Amount'
                ]
                members_cols = [
                    'id', 'first_name', 'last_name',  'full_name','email', 'cell_phone', 'membership_status', 'membership_type',
                    'date_paid','payment_received','member_id','source_file','mailchimp_id','last_synced'
                ]
                # Only export the selected columns from above
                if len(matched_df) > 0:
                    # Ensure Amount and Last_Payment_Amount are numeric for formatting to work
                    matched_df['Amount'] = pd.to_numeric(matched_df['Amount'], errors='coerce')
                    matched_df['Last_Payment_Amount'] = pd.to_numeric(matched_df['Last_Payment_Amount'], errors='coerce')
                    matched_df[matched_cols].to_excel(writer, sheet_name='Matched Transactions', index=False)

                if len(unmatched_df) > 0:
                    unmatched_df[unmatched_cols].to_excel(writer, sheet_name='Unmatched Transactions', index=False)
                
                if 'members_df' in globals() and len(members_df) > 0:
                    # Add status priority column for custom sorting
                    status_priority_map = {
                        'Paid Up Member': 1,
                        'Unpaid Member': 2,
                        'Subscribed': 3,
                        'Contact': 4,
                        'Cleaned': 5,
                        'Pending': 6,
                        'WhatsApp': 7
                    }
                    # Change the sort order here
                    members_df['status_priority'] = members_df['membership_status'].map(status_priority_map).fillna(99)
                    # Sort by full_name (A-Z), then by status_priority (lowest number = highest priority)
                    members_df_sorted = members_df.sort_values(
                        by=['full_name', 'status_priority'],
                        ascending=[True, True]
                    )
                    members_df_sorted[members_cols].to_excel(writer, sheet_name='All Members', index=False)
                
                workbook = writer.book
                # --- Add currency formatting ---
                currency_format = workbook.add_format({'num_format': 'R #,##0.00'})
                # Format 'Amount' and 'Last_Payment_Amount' in Matched Transactions
                if 'Matched Transactions' in writer.sheets:
                    ws = writer.sheets['Matched Transactions']
                    amt_col = matched_cols.index('Amount')
                    last_amt_col = matched_cols.index('Last_Payment_Amount')
                    ws.set_column(amt_col, amt_col, 12, currency_format)
                    ws.set_column(last_amt_col, last_amt_col, 18, currency_format)
                # Format 'Amount' in Unmatched Transactions
                if 'Unmatched Transactions' in writer.sheets:
                    ws = writer.sheets['Unmatched Transactions']
                    amt_col = unmatched_cols.index('Amount')
                    ws.set_column(amt_col, amt_col, 12, currency_format)
                # Format 'Last_Payment_Amount' in All Members (if present)
                if 'All Members' in writer.sheets and 'Last_Payment_Amount' in members_cols:
                    ws = writer.sheets['All Members']
                    last_amt_col = members_cols.index('Last_Payment_Amount')
                    ws.set_column(last_amt_col, last_amt_col, 18, currency_format)
                # Autofilter and freeze panes
                for sheet in writer.sheets:
                    worksheet = writer.sheets[sheet]
                    worksheet.autofilter(0, 0, worksheet.dim_rowmax, worksheet.dim_colmax)
                    worksheet.freeze_panes(1, 0)
            print(f"✅ Exported results to: {output_path}")
            
            # Standardize output path display
            rel_output = output_path.replace('\\', '/')
            marker = '/web-based-membership-management-system/data_output/'
            idx = rel_output.find(marker)
            if idx != -1:
                rel_output = rel_output[idx + 1:] if rel_output[idx] == '/' else rel_output[idx:]
            print(f"📁 File saved: {rel_output}")
            if auto_open:
                os.startfile(output_path)

        except Exception as e:
            print(f"❌ Failed to export Excel file: {str(e)}")

    return matched_df, unmatched_df

# Usage example:
matched_df, unmatched_df = process_transactions(df, export_excel=True, auto_open=True)

(240, 4)
       Date  Amount                    Type                     Description
0  20250131   200.0         IB PAYMENT FROM            COGHO - OCTOBER 2023
1  20250131   -46.7             SERVICE FEE                             NaN
2  20250131   -95.0  MONTHLY MANAGEMENT FEE                             NaN
3  20250203  -149.0         ACCOUNT PAYMENT  MULTID FORXNEELO      85541122
4  20250205 -1044.3           IB PAYMENT TO             BEN TROMP ONDERHOUD
Function started

Total members loaded: 2049

Processing results:
Matched: 115 | Unmatched: 125
✅ Exported results to: C:/Work/active_projects/web-based-membership-management-system\data_output\transaction_matching_simple_20250809_143135.xlsx
📁 File saved: web-based-membership-management-system/data_output/transaction_matching_simple_20250809_143135.xlsx
